In [2]:
import gymnasium as gym
import numpy as np
from stable_baselines3.common.env_util import make_vec_env
from copy import deepcopy
import torch
from torch.distributions import Normal
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.data import Dataset, DataLoader
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import imageio
import pandas as pd
from tqdm import tqdm
from PIL import Image
from stable_baselines3.dqn.policies import MlpPolicy
import matplotlib.pyplot as plt


In [9]:
import gc
gc.collect()

2428

In [10]:
!nvidia-smi
# clean up memory forcefully
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device



Tue Apr  2 01:06:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
|  8%   52C    P2              57W / 250W |    250MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

device(type='cuda')

In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
class CustomDataset(Dataset):
    def __init__(self, observations, actions):
        self.observations = torch.tensor(observations, dtype=torch.float32,device=device)
        self.actions = torch.tensor(actions, dtype=torch.float32,device=device)

    def __len__(self):
        return len(self.observations)

    def __getitem__(self, idx):
        observation = self.observations[idx]
        action = self.actions[idx]
        return observation, action

class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    

Using device: cuda


In [5]:
env_id = "CartPole-v1"
# env = make_vec_env(env_id, n_envs=1)
# read /mnt/nfs/work/c98181/RL/dataset/CartPole-v1...npy
observations= np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_1M_obs.npy", allow_pickle=True)
actions = np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_1M_actions.npy", allow_pickle=True)
observations=observations.squeeze()
actions=actions.squeeze()
# observations = observations[:1000]
# actions = actions[:1000]
print(observations.shape, actions.shape)
print(observations[0], actions[0])

# print(actions)

(1000000, 4) (1000000,)
[-0.03013654  0.21001437 -0.04612853 -0.34675622] 1


In [6]:
class mydataset(Dataset):
    def __init__(self, observations, actions):
        self.observations = torch.tensor(observations, dtype=torch.float32).to(device)
        self.actions = torch.tensor(actions, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.observations)

    def __getitem__(self, idx):
        observation = self.observations[idx]
        action = self.actions[idx]
        return observation, action

In [ ]:
policy_network = PolicyNetwork(
    4, 2).to(device)

prev = PolicyNetwork(
    4, 2).to(device)


optimizer = torch.optim.Adam(policy_network.parameters(), lr=(1e-3))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)


# Training loop
num_epochs = 300
batch_size = 512

dataset = mydataset(observations=observations, actions=actions)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

graph=[]
eval_rewards=[]
for epoch in range(num_epochs):
    policy_network.train()

    # Compute the log probabilities of the actions
    pbar=tqdm(dataloader,position=0,leave=True)
    loss_record=[]
    # step=0
    for step, (obs_batch,act_batch) in enumerate(pbar):
        # obs_batch = torch.tensor(np.array(obs_batch), dtype=torch.float32).to(device)
        # act_batch = torch.tensor(np.array(act_batch), dtype=torch.float32).to(device)
        if step % 100 == 0:
          prev.load_state_dict(policy_network.state_dict())
          prev.eval()
        # Get the log probabilities of the actions
        
        softmax = policy_network(obs_batch)
        # print(softmax.shape, act_batch.shape)
        model_act_sample = Categorical(softmax).sample()
        policy_chosen_logps = Categorical(softmax).log_prob(act_batch)
        policy_rejected_logps = Categorical(softmax).log_prob(model_act_sample)
        with torch.no_grad():
            reference_chosen_logps = Categorical(prev(obs_batch)).log_prob(act_batch)
            reference_rejected_logps = Categorical(prev(obs_batch)).log_prob(model_act_sample)
          

        pi_logratios = policy_chosen_logps - policy_rejected_logps
        ref_logratios = reference_chosen_logps - reference_rejected_logps

        # chosen_KL = (policy_chosen_logps - reference_chosen_logps).mean().clamp(min=0)
        # reject_KL = (policy_rejected_logps - reference_rejected_logps).mean().clamp(min=0)

        logits = pi_logratios - ref_logratios

        chosen_logratios = policy_chosen_logps - reference_chosen_logps
        reject_logratios = policy_rejected_logps - reference_rejected_logps

        # logits = chosen_logratios - reject_logratios

        if epoch <= 10:
          loss = - (policy_chosen_logps).mean()
        else:
          beta = 1
          losses = (
                  -F.logsigmoid(beta * logits)
              )
          # loss = losses.mean()
        #   losses = torch.cat((1 - F.logsigmoid(beta * (chosen_logratios - reject_KL)), 1 - F.logsigmoid(beta * (chosen_KL - reject_logratios))), 0)
          loss = losses.mean()
        # Optimize the policy
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
        pbar.set_description((f"Epoch [{epoch+1}/{num_epochs}]"))
        positive_reward = chosen_logratios.detach().mean().item()
        negative_reward = reject_logratios.detach().mean().item()
        pbar.set_postfix({"loss":loss.detach().item(), "positive_reward": positive_reward, "negative_reward": negative_reward, "margin": positive_reward - negative_reward})


        #scheduler

        loss_record.append(loss.detach().item())
    scheduler.step()
    graph.append(sum(loss_record)/len(loss_record))

    policy_network.eval()  # 切换到评估模式

    env = make_vec_env(env_id, n_envs=1)

    state = env.reset()
    done = False
    total_reward=0
    # test the policy and save as gif
    frames = []
    while not done:
        state_tensor = torch.tensor([state], dtype=torch.float32).to(device)
        with torch.no_grad():
            action = Categorical(policy_network(state_tensor)).sample().cpu().numpy()[0]
            
        state, reward, done, _ = env.step(action)  # 执行动作
        total_reward += reward
        frame = env.render(mode="rgb_array")  # 获取当前环境的图像
        frames.append(frame)  # 添加到帧列表中

    # 保存为GIF
    image_path = f"cartpole_epoch_{epoch+1}.gif"
    imageio.mimsave("/mnt/nfs/work/c98181/RL/result/"+env_id+"/"+image_path, frames, duration=0.04)  # duration控制帧切换的速度


    env.close()
    print(total_reward)
    eval_rewards.append(total_reward)

In [ ]:
# draw the loss graph
import matplotlib.pyplot as plt
plt.plot(graph)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch")
plt.show()

# draw the reward graph
plt.plot(eval_rewards)
plt.xlabel("Epoch")
plt.ylabel("Reward")
plt.title("Reward vs Epoch")
plt.show()



In [11]:
# show line number below


policy_network = PolicyNetwork(
    4, 2).to(device)
# nn init kaiming
torch.nn.init.kaiming_normal_(policy_network.fc1.weight)
torch.nn.init.kaiming_normal_(policy_network.fc2.weight)
torch.nn.init.kaiming_normal_(policy_network.fc3.weight)

# build training set and validation set
train_size = int(0.9 * len(observations))
train_observations = observations[:train_size]
train_actions = actions[:train_size]
val_observations = observations[train_size:]
val_actions = actions[train_size:]

# build dataset
train_dataset = CustomDataset(train_observations, train_actions)
val_dataset = CustomDataset(val_observations, val_actions)


optimizer = torch.optim.Adam(policy_network.parameters(), lr=(5e-4),weight_decay=1e-4,eps=1e-8)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)


# Training loop
num_epochs = 1000
batch_size = 1024

train_dataloader = DataLoader(train_dataset , batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset , batch_size=batch_size, shuffle=True)
loss_func = nn.CrossEntropyLoss()

graph=[]
val_losses=[]
eval_rewards=[]
for epoch in range(num_epochs):

        



    policy_network.train()

    # Compute the log probabilities of the actions
    pbar=tqdm(train_dataloader,position=0,leave=True)
    loss_record=[]

    for step, (obs_batch,act_batch) in enumerate(pbar):

        
        logits = policy_network(obs_batch)
        # cross entropy
        loss=loss_func(logits, act_batch.long())
        
        # Optimize the policy
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        pbar.set_description((f"Epoch [{epoch+1}/{num_epochs}]"))
        pbar.set_postfix({"loss":loss.detach().item()})


        #scheduler

        loss_record.append(loss.detach().item())
    scheduler.step()
    graph.append(sum(loss_record)/len(loss_record))
    with torch.no_grad():
        # validation_loss
        val_loss_record=[]
        for step, (obs_batch,act_batch) in enumerate(val_dataloader):
            logits = policy_network(obs_batch)
            loss = loss_func(logits, act_batch.long())
            val_loss_record.append(loss.detach().item())
        val_loss = sum(val_loss_record)/len(val_loss_record)
        val_losses.append(val_loss)
    
    policy_network.eval()  # 切换到评估模式

    env = make_vec_env(env_id, n_envs=1)
    # test ten times
    total_reward=0
    for _ in range(10):
        state = env.reset()
        done = False
        
        # test the policy and save as gif
        frames = []
        while not done:
            state_tensor = torch.tensor([state], dtype=torch.float32).to(device)
            with torch.no_grad():
                pred= policy_network(state_tensor)
                action = torch.argmax(pred).cpu().numpy()
                # print(action)
                
            state, reward, done, _ = env.step([action])  # 执行动作
            total_reward += reward
            frame = env.render(mode="rgb_array")  # 获取当前环境的图像
            frames.append(frame)  # 添加到帧列表中

    # 保存为GIF
    image_path = f"cartpole_epoch_{epoch+1}.gif"
    imageio.mimsave("/mnt/nfs/work/c98181/RL/result/"+env_id+"/gif/"+image_path, frames, duration=0.04)  # duration控制帧切换的速度


    env.close()
    # print(total_reward/10, val_loss)
    
    eval_rewards.append(total_reward/10)
    if (epoch+1) % 50 == 0:
        print("saving model")
        torch.save(policy_network.state_dict(), f"/mnt/nfs/work/c98181/RL/result/{env_id}/model/policy_network_epoch_{epoch+1}.pth")
        # plot the loss graph
        plt.plot(graph)
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Loss vs Epoch")
        plt.savefig(f"/mnt/nfs/work/c98181/RL/result/{env_id}/graph/loss_epoch_{epoch+1}.png")
        plt.close()
        # plot the reward graph
        plt.plot(eval_rewards)
        plt.xlabel("Epoch")
        plt.ylabel("Reward")
        plt.title("Reward vs Epoch")
        plt.savefig(f"/mnt/nfs/work/c98181/RL/result/{env_id}/graph/reward_epoch_{epoch+1}.png")
        plt.close()
        # plot the val loss graph
        plt.plot(val_losses)
        plt.xlabel("Epoch")
        plt.ylabel("Val Loss")
        plt.title("Val Loss vs Epoch")
        plt.savefig(f"/mnt/nfs/work/c98181/RL/result/{env_id}/graph/val_loss_epoch_{epoch+1}.png")
        plt.close()

Epoch [50/1000]: 100%|██████████| 879/879 [00:07<00:00, 119.65it/s, loss=0.113] 


saving model


Epoch [100/1000]: 100%|██████████| 879/879 [00:07<00:00, 122.62it/s, loss=0.0953]


saving model


Epoch [150/1000]: 100%|██████████| 879/879 [00:07<00:00, 122.05it/s, loss=0.111] 


...

Epoch [650/1000]: 100%|██████████| 879/879 [00:07<00:00, 122.59it/s, loss=0.101] 


saving model


Epoch [700/1000]: 100%|██████████| 879/879 [00:07<00:00, 123.85it/s, loss=0.1]   


saving model


Epoch [732/1000]:  32%|███▏      | 277/879 [00:02<00:06, 96.54it/s, loss=0.0957] 

In [ ]:
# draw the loss graph
import matplotlib.pyplot as plt
plt.plot(graph)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch")
plt.show()

# draw the reward graph
plt.plot(eval_rewards)
plt.xlabel("Epoch")
plt.ylabel("Reward")
plt.title("Reward vs Epoch")
plt.show()
# draw the validation loss graph
plt.plot(val_losses)
plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.title("Validation Loss vs Epoch")
plt.show()



In [ ]:
# def pretrain(self, dataset, n_epochs=10, learning_rate=1e-4,
#                  adam_epsilon=1e-8, val_interval=None):
#         """
#         Pretrain a model using behavior cloning:
#         supervised learning given an expert dataset.

#         NOTE: only Box and Discrete spaces are supported for now.

#         :param dataset: (ExpertDataset) Dataset manager
#         :param n_epochs: (int) Number of iterations on the training set
#         :param learning_rate: (float) Learning rate
#         :param adam_epsilon: (float) the epsilon value for the adam optimizer
#         :param val_interval: (int) Report training and validation losses every n epochs.
#             By default, every 10th of the maximum number of epochs.
#         :return: (BaseRLModel) the pretrained model
#         """
#         continuous_actions = isinstance(self.action_space, gym.spaces.Box)
#         discrete_actions = isinstance(self.action_space, gym.spaces.Discrete)

#         assert discrete_actions or continuous_actions, 'Only Discrete and Box action spaces are supported'

#         # Validate the model every 10% of the total number of iteration
#         if val_interval is None:
#             # Prevent modulo by zero
#             if n_epochs < 10:
#                 val_interval = 1
#             else:
#                 val_interval = int(n_epochs / 10)

#         with self.graph.as_default():
#             with tf.variable_scope('pretrain', reuse=tf.AUTO_REUSE):
#                 if continuous_actions:
#                     obs_ph, actions_ph, deterministic_actions_ph = self._get_pretrain_placeholders()
#                     loss = tf.reduce_mean(tf.square(actions_ph - deterministic_actions_ph))
#                 else:
#                     obs_ph, actions_ph, actions_logits_ph = self._get_pretrain_placeholders()
#                     # actions_ph has a shape if (n_batch,), we reshape it to (n_batch, 1)
#                     # so no additional changes is needed in the dataloader
#                     actions_ph = tf.expand_dims(actions_ph, axis=1)
#                     one_hot_actions = tf.one_hot(actions_ph, self.action_space.n)
#                     loss = tf.nn.softmax_cross_entropy_with_logits_v2(
#                         logits=actions_logits_ph,
#                         labels=tf.stop_gradient(one_hot_actions)
#                     )
#                     loss = tf.reduce_mean(loss)
#                 optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=adam_epsilon)
#                 optim_op = optimizer.minimize(loss, var_list=self.params)

#             self.sess.run(tf.global_variables_initializer())

#         if self.verbose > 0:
#             print("Pretraining with Behavior Cloning...")

#         for epoch_idx in range(int(n_epochs)):
#             train_loss = 0.0
#             # Full pass on the training set
#             for _ in range(len(dataset.train_loader)):
#                 expert_obs, expert_actions = dataset.get_next_batch('train')
#                 feed_dict = {
#                     obs_ph: expert_obs,
#                     actions_ph: expert_actions,
#                 }
#                 train_loss_, _ = self.sess.run([loss, optim_op], feed_dict)
#                 train_loss += train_loss_

#             train_loss /= len(dataset.train_loader)

#             if self.verbose > 0 and (epoch_idx + 1) % val_interval == 0:
#                 val_loss = 0.0
#                 # Full pass on the validation set
#                 for _ in range(len(dataset.val_loader)):
#                     expert_obs, expert_actions = dataset.get_next_batch('val')
#                     val_loss_, = self.sess.run([loss], {obs_ph: expert_obs,
#                                                         actions_ph: expert_actions})
#                     val_loss += val_loss_

#                 val_loss /= len(dataset.val_loader)
#                 if self.verbose > 0:
#                     print("==== Training progress {:.2f}% ====".format(100 * (epoch_idx + 1) / n_epochs))
#                     print('Epoch {}'.format(epoch_idx + 1))
#                     print("Training loss: {:.6f}, Validation loss: {:.6f}".format(train_loss, val_loss))
#                     print()
#             # Free memory
#             del expert_obs, expert_actions
#         if self.verbose > 0:
#             print("Pretraining done.")
#         return self
